In [6]:
import numpy as np
from scipy.linalg.interpolative import interp_decomp
import matplotlib.pyplot as plt

In [7]:
# Use scipy's implementation of the interpolative decomposition
# Instead of the matrix cross interpolation M = C @ P^-1 @ R
# it factorizes as M = A @ P with A = M[:, idx]
def interpolative_decomposition(M, eps_or_k=1e-5, k_min=2):
    r = min(M.shape)
    if r <= k_min:
        k = r
        idx, proj = interp_decomp(M, eps_or_k=k) #eps_or_k = precision of decomposition
    elif isinstance(eps_or_k, int): #checks if eps is an integer
        k = min(r, eps_or_k)
        idx, proj = interp_decomp(M, eps_or_k=k)
    else:
        k, idx,  proj = interp_decomp(M, eps_or_k=eps_or_k)
        if k <= k_min:
            k = min(r, k_min) #is it not enough to put k = k_min? 
                              #r>k_min otherwise first condition would have been true
            idx, proj = interp_decomp(M, eps_or_k=k)
    A = M[:, idx[:k]]
    P = np.concatenate([np.eye(k), proj], axis=1)[:, np.argsort(idx)]
    return A, P, k, idx[:k]

# k is the 'compressed' rank = number of pivot columns
# idx is the array with entries the indeces of the pivot columns
# proj = matrix R s.t. M[:,idx[:k]]*R = M[:,idx[k:]] 
# P = matrix s.t.  M[:,idx[:k]]*P = M (approximated)

In [11]:
class function:  # certain function f(x) with x given as binary

    def __init__(self, f):
        self.cache = {}
        self.f = f #store function passed during instantiation


    def __call__(self, *args, **kwds):
        
        if args in self.cache:
            return self.cache[args]
        else:
            val = self.f(*args)
            self.cache[*args] = val
            return val
    
    def cache_size(self): #size of cache = number of current evaluations
        return len(self.cache) #return the number of entries in the cache
        


### FIRST VERSION

In [ ]:
# implement the tensor cross interpolation
def tensor_cross_interpolation(tensor, func_vals, L, d=2, eps_or_chi=1e-6, iters=6):
    # random initial choice for index sets
    idxs = np.random.choice(d, size=(L)) #array of L random numbers from 0 to d-1 - index sigma
    As = [np.array([[[tensor(*idxs[:j], i, *idxs[j+1:])] for i in range(d)]])
          for j in range(L)]
    As[1:] /= tensor(*idxs)
    #this sort of corresponds to inserting the P^-1 0-dimensional slices
    #i.e. if I don't put this, evaluating F(σ1,...,σL)= As*...*As = F(σ1,..,σL)^L, when it should give just F!
    eval = []
    err_max = []
    err_2 = []
    I = [idxs[:j].reshape(1, -1) for j in range(L)] # creates list of I_l arrays
    J = [idxs[j:].reshape(1, -1) for j in range(1, L+1)] # list of J_l
    # sweep
    for i in range(iters):
        #print(f'Sweep: {i+1:d}.')
        As, I = left_to_right_sweep(tensor, As, I, J, L, d, eps_or_chi)
        As, J = right_to_left_sweep(tensor, As, I, J, L, d, eps_or_chi)
    return As, J, eval

def left_to_right_sweep(tensor, func_vals, As, I, J, L, d, eps_or_chi, eval):
    # sweep left to right
    for bond in range(L-1):
        # construct local two-site tensor
        chil, _ = I[bond].shape #chil = number of rows in array I_l: number of combinations (σ1,..,σl)
        chir, _ = J[bond+1].shape #which corresponds to number of "points" on which I am evaluating function

        Pi = np.zeros(chil, d, d, chir)
        for il in range(chil):
            for s1 in range(d):
                for s2 in range(d):
                    for jr in range(chir):
                        val = tensor(*I[bond][il,:],s1,s2,*J[bond][jr,:])
                        Pi[il,s1,s2,jr] = val
        # decompose using interpolative decomposition:
        # Pi = P^T @ A^T, A^T = Pi[idx,:]
        A, P, k, idx = interpolative_decomposition(Pi.T, eps_or_k=eps_or_chi)
        # update indices using idxs c I[bond] x {0, 1, ..., d-1}
        I[bond+1] = np.array([np.append(I[bond][i//d], [i%d]) for i in idx])
        # update tensors
        As[bond] = P.T.reshape(chil, d, k)
        As[bond+1] = A.T.reshape(k, d, chir)
        eval.append(tensor.cache_size())

        if bond==0:
            func_interp = np.squeeze(As[0])
            func_updated = func_interp
            for A in As[1:]:
                func_interp = np.einsum('ia, ajb -> ijb', func_interp, A)
                func_interp = func_interp.reshape(-1, A.shape[-1])
        else: #should work just with this because the for cycle goes up to bond=L-2
            func_updated = np.einsum('ia, ajb -> ijb', func_updated, As[bond])
            func_interp = func_updated
            for A in As[(bond+1):]:
                func_interp =  np.einsum('ia, ajb -> ijb', func_interp, A)
                func_interp = func_interp.reshape(-1, A.shape[-1])
        """elif bond == L-1:
            func_updated = np.einsum('ia, ajb -> ijb', func_updated, As[bond])
            func_interp = func_updated
            func_interp = func_interp.reshape(-1, A.shape[-1])"""


        func_interp = np.squeeze(func_interp)

        difference = func_vals-func_interp
        err_max.append(np.max(np.abs(difference)))
        err_2.append(np.linalg.norm(difference)**2)

    return As, I

def right_to_left_sweep(tensor, func_vals, As, I, J, L, d, eps_or_chi):
    # sweep right to left
    for bond in range(L-2,-1,-1):
        # construct local two-site tensor
        chil, _ = I[bond].shape
        chir, _ = J[bond+1].shape
        Pi = np.zeros(chil, d, d, chir)
        for il in range(chil):
            for s1 in range(d):
                for s2 in range(d):
                    for jr in range(chir):
                        val = tensor(*I[bond][il,:],s1,s2,*J[bond][jr,:])
                        Pi[il,s1,s2,jr] = val

        # decompose using interpolative decomposition:
        # Pi = A @ P, A = Pi[:,idx]
        A, P, k, idx = interpolative_decomposition(Pi, eps_or_k=eps_or_chi)
        # update indices using idxs c {0, 1, ..., d-1} x J[bond+1]
        J[bond] = np.array([np.append([i//chir], J[bond+1][i%chir]) for i in idx])
        # update tensors
        As[bond] = A.reshape(chil, d, k)
        As[bond+1] = P.reshape(k, d, chir)
        eval.append(tensor.cache_size())

        if bond == L-2:
        As_new = np.einsum('ia, ajb -> ijb', As[] As[bond+1])

        

        difference = func_vals-func_interp
        err_max.append(np.max(np.abs(difference)))
        err_2.append(np.linalg.norm(difference)**2)

    return As, J

### SECOND VERSION

In [ ]:
# implement the tensor cross interpolation
def tensor_cross_interpolation(tensor, func_vals, L, d=2, eps_or_chi=1e-6, iters=6):
    # random initial choice for index sets
    idxs = np.random.choice(d, size=(L)) #array of L random numbers from 0 to d-1 - index sigma
    As = [np.array([[[tensor(*idxs[:j], i, *idxs[j+1:])] for i in range(d)]])
          for j in range(L)]
    As[1:] /= tensor(*idxs)
    #this sort of corresponds to inserting the P^-1 0-dimensional slices
    #i.e. if I don't put this, evaluating F(σ1,...,σL)= As*...*As = F(σ1,..,σL)^L, when it should give just F!
    eval = []
    err_max = []
    err_2 = []
    I = [idxs[:j].reshape(1, -1) for j in range(L)] # creates list of I_l arrays
    J = [idxs[j:].reshape(1, -1) for j in range(1, L+1)] # list of J_l
    # sweep
    As_new = 0
    for i in range(iters):
        #print(f'Sweep: {i+1:d}.')
        As, I, func_updated = left_to_right_sweep(tensor, func_vals, As_new, As, I, J, L, d, eps_or_chi)
        As, J, As_new = right_to_left_sweep(tensor, func_vals, func_updated, As, I, J, L, d, eps_or_chi)
    return As, J, eval, err_2, err_max

def left_to_right_sweep(tensor, func_vals, As_new, As, I, J, L, d, eps_or_chi, eval):
    # sweep left to right
    func_updated = []
    for bond in range(L-1):
        # construct local two-site tensor
        chil, _ = I[bond].shape #chil = number of rows in array I_l: number of combinations (σ1,..,σl)
        chir, _ = J[bond+1].shape #which corresponds to number of "points" on which I am evaluating function

        Pi = np.zeros(chil, d, d, chir)
        for il in range(chil):
            for s1 in range(d):
                for s2 in range(d):
                    for jr in range(chir):
                        val = tensor(*I[bond][il,:],s1,s2,*J[bond][jr,:])
                        Pi[il,s1,s2,jr] = val
        # decompose using interpolative decomposition:
        # Pi = P^T @ A^T, A^T = Pi[idx,:]
        A, P, k, idx = interpolative_decomposition(Pi.T, eps_or_k=eps_or_chi)
        # update indices using idxs c I[bond] x {0, 1, ..., d-1}
        I[bond+1] = np.array([np.append(I[bond][i//d], [i%d]) for i in idx])
        # update tensors
        As[bond] = P.T.reshape(chil, d, k)
        As[bond+1] = A.T.reshape(k, d, chir)
        eval.append(tensor.cache_size())

        if As_new == 0:
            if bond==0:
                func_interp = np.squeeze(As[0])
                func_updated.append(func_interp)
                for A in As[1:]:
                    func_interp = np.einsum('ia, ajb -> ijb', func_interp, A)
                    func_interp = func_interp.reshape(-1, A.shape[-1])
            else: #should work just with this because the for cycle goes up to bond=L-2
                func_interp = np.einsum('ia, ajb -> ijb', func_updated[-1], As[bond])
                func_updated.append(func_interp) #I should fill the list up to bond=L-2, so the list should have L-1 elements
                for A in As[(bond+1):]:
                    func_interp =  np.einsum('ia, ajb -> ijb', func_interp, A)
                    func_interp = func_interp.reshape(-1, A.shape[-1])
            """elif bond == L-1:
            func_updated = np.einsum('ia, ajb -> ijb', func_updated, As[bond])
            func_interp = func_updated
            func_interp = func_interp.reshape(-1, A.shape[-1])"""
        else:
            if bond==0:
                func_interp = np.squeeze(As[0])
                func_updated.append(func_interp)
                func_interp = np.einsum('ia, ajb -> ijb', func_interp, As[1])
                func_interp = func_interp.reshape(-1, A.shape[-1])
                func_interp = np.einsum('ia, ajb -> ijb', func_interp, As_new[0])
                func_interp = func_interp.reshape(-1, A.shape[-1]) 
        #for now this part is not compatible with "else" part
            else: #should work just with this because the for cycle goes up to bond=L-2
                func_interp = np.einsum('ia, ajb -> ijb', func_updated[-1], As[bond])
                func_updated.append(func_interp)
                func_interp = np.einsum('ia, ajb -> ijb', func_interp, As_new[bond])
                func_interp = func_interp.reshape(-1, A.shape[-1])

        func_interp = np.squeeze(func_interp)

        difference = func_vals-func_interp
        err_max.append(np.max(np.abs(difference)))
        err_2.append(np.linalg.norm(difference)**2)

    return As, I, func_updated

def right_to_left_sweep(tensor, func_vals, func_updated, As, I, J, L, d, eps_or_chi):
    # sweep right to left
    As_updated = []
    for bond in range(L-2,-1,-1):
        # construct local two-site tensor
        chil, _ = I[bond].shape
        chir, _ = J[bond+1].shape
        Pi = np.zeros(chil, d, d, chir)
        for il in range(chil):
            for s1 in range(d):
                for s2 in range(d):
                    for jr in range(chir):
                        val = tensor(*I[bond][il,:],s1,s2,*J[bond][jr,:])
                        Pi[il,s1,s2,jr] = val

        # decompose using interpolative decomposition:
        # Pi = A @ P, A = Pi[:,idx]
        A, P, k, idx = interpolative_decomposition(Pi, eps_or_k=eps_or_chi)
        # update indices using idxs c {0, 1, ..., d-1} x J[bond+1]
        J[bond] = np.array([np.append([i//chir], J[bond+1][i%chir]) for i in idx])
        # update tensors
        As[bond] = A.reshape(chil, d, k)
        As[bond+1] = P.reshape(k, d, chir)
        eval.append(tensor.cache_size())

        if bond == L-2:
            As_new = As[L-1]
            As_new = np.einsum('ia, ajb -> ijb', As[bond], As_new)
            As_updated.append(As_new)
            func_interp = np.einsum('ia, ajb -> ijb', func_updated[-2], As_new)
            func_interp = func_interp.reshape(-1, A.shape[-1])
        else:
            As_new = np.einsum('ia, ajb -> ijb', As[bond], As_new)
            As_updated.append(As_new)
            func_interp = np.einsum('ia, ajb -> ijb', func_updated[bond-1], As_new)
            func_interp = func_interp.reshape(-1, A.shape[-1])

        func_interp = np.squeeze(func_interp)

        difference = func_vals-func_interp
        err_max.append(np.max(np.abs(difference)))
        err_2.append(np.linalg.norm(difference)**2)

    return As, J, As_new

In [15]:
A = [0,1,2,3,4]

for Ar in A[::-1]:
    print(Ar)

B = []

for i in range(4):
    B.append(i)

print(B)

4
3
2
1
0
[0, 1, 2, 3]


In [7]:
def eval_error(test_func, xs, maxiter):
    func_vals = test_func(xs)
    func = lambda *args: func_vals.reshape((2,)*10)[*args]
    evals = []
    err_max = []
    err_2 = []
    As = []
    for iters in range(1,maxiter+1):
        As.append(tensor_cross_interpolation(func, L = 10, eps_or_chi=10, iters=iters)[0])
        evals.append(tensor_cross_interpolation(func, L = 10, eps_or_chi=10, iters=iters)[2])
    for i in range(maxiter):
        func_interp = np.squeeze(As[i][0])
        for A in As[i][1:]:
            func_interp = np.einsum('ia, ajb -> ijb', func_interp, A)
            func_interp = func_interp.reshape(-1, A.shape[-1])
        func_interp = np.squeeze(func_interp)
        difference = func_vals-func_interp
        plt.plot(xs,func_vals, '-')
        plt.plot(xs,func_interp, '--')
        plt.show()
        err_max.append(np.max(np.abs(difference)))
        err_2.append(np.linalg.norm(difference)**2)

    plt.plot(evals,err_2, '-', label = "$\epsilon_2$")
    plt.plot(evals,err_max, '--', label = "$\epsilon_\infty$")
    plt.xlabel("evals")
    plt.ylabel("error")
    plt.legend()
    plt.show()


    return err_max, err_2, evals



<>:24: SyntaxWarning: invalid escape sequence '\e'
<>:25: SyntaxWarning: invalid escape sequence '\e'
<>:24: SyntaxWarning: invalid escape sequence '\e'
<>:25: SyntaxWarning: invalid escape sequence '\e'
/tmp/ipykernel_3095888/2226458619.py:24: SyntaxWarning: invalid escape sequence '\e'
  plt.plot(evals,err_2, '-', label = "$\epsilon_2$")
/tmp/ipykernel_3095888/2226458619.py:25: SyntaxWarning: invalid escape sequence '\e'
  plt.plot(evals,err_max, '--', label = "$\epsilon_\infty$")


In [ ]:
# Example: create a function to interpolate
test_func = lambda x: 0.4 * np.sin(10 * np.pi * x)\
                    + 0.1 * np.cos(99 * np.pi * x)\
                    + 4 * x * (1 - x)

# exact function values
xs = np.linspace(0, 1, 2**10, endpoint=False)
func_vals = test_func(xs)

# MPS from tensor cross interpolation
#func = lambda *args: func_vals.reshape((2,)*10)[*args]
#here we have to define func as the initialization of the class
func = function(lambda x: func_vals.reshape((2,) * 10)[*x])
As, _, eval = tensor_cross_interpolation(func, func_vals          # function to be interpolated
                                   L=10,          # number of MPS tensors
                                   eps_or_chi=10, # bond dimension
                                   iters=4)       # number of back-and-forth sweeps


TypeError: tensor_cross_interpolation() missing 1 required positional argument: 'func_vals'